In [32]:
import feedparser
from openai import OpenAI 

client = OpenAI(api_key = OPEN_API_KEY)

# URL of the RSS feed
rss_url = "https://www.smh.com.au/rss/business.xml"

def fetch_rss_entries(rss_url):
    # Parse the RSS feed
    feed = feedparser.parse(rss_url)
    # print(feed)
    entries = feed.entries
    return entries

def get_title_summary(entries):
    entries_list = []
    for entry in entries:
        dict = {}
        dict['title'] = entry.title 
        dict['summary'] = entry.summary
        entries_list.append(dict)
    return(entries_list)

def summarize_news_with_chatgpt(articles):        
    prompt = (
        "Summarize the following financial news articles in a concise manner (without special characters), highlighting the key stories which can move the Australian and APAC (Asia Pacific) markets:\n\n" 
        + str(articles)
    )
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o",  # Use GPT-4o-mini/gpt-3.5-turbo for the latest model
            messages=[
                {"role": "system", "content": "You are a financial expert summarizing daily financial news for the Australian and APAC (Asia Pacific) stock markets."},
                {"role": "user", "content": prompt},
            ],
            max_tokens=900,
            temperature=0.5,
        )
        return response.choices[0].message.content        
        # return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        return f"Error: {e}"

if __name__ == "__main__":
    # Fetch the RSS feed entries
    entries = fetch_rss_entries(rss_url)

    # print(entries)


    # # Summarize the news using ChatGPT
    if entries:
        title_summaries = get_title_summary(entries)
        summary = summarize_news_with_chatgpt(title_summaries)
        print("Summary of the day's financial news:")
        print(summary)
    else:
        print("No news entries found.")


Summary of the day's financial news:
1. **ASX Edges Up Ahead of Key Jobs Report**:
   The Australian stock market (ASX) saw a modest rise as Wall Street experienced mixed trading, with indexes slipping ahead of a crucial employment data release.

2. **China Faces Economic Challenges**:
   China is grappling with significant economic issues as consumer spending declines and investors withdraw billions, potentially leading to a prolonged period of economic stagnation.

3. **ASX Gains Despite US Market Losses**:
   The ASX closed higher, buoyed by gains in IT and property stocks, despite a slowdown in losses on Wall Street following a significant sell-off.

4. **Lincoln Minerals Explores Uranium in SA**:
   Lincoln Minerals has commenced exploration for uranium at the Yallunda project in South Australia, targeting a 7km-long surface anomaly.

5. **Interest Rates Likely to Hold Steady in Australia**:
   The Reserve Bank of Australia is expected to keep interest rates unchanged, viewing the

In [7]:
entries[0].summary

'With the antimony price surging, Dart Mining has revived its hunt for the metal at its Victorian gold projects in a region with a rich history for the commodity.'

In [11]:
title_summaries = get_title_summary(entries)


In [12]:
title_summaries

[{'title': 'Dart revives hot commodity antimony hunt in Victoria',
  'summary': 'With the antimony price surging, Dart Mining has revived its hunt for the metal at its Victorian gold projects in a region with a rich history for the commodity.'},
 {'title': 'ASX drops sharply as Wall Street dives; Miners, banks, tech stocks decline',
  'summary': 'The Australian sharemarket is slumping after Wall Street had its worst session since the August 5 market meltdown amid concerns about a slowdown in the world’s largest economy.'},
 {'title': 'Hastings posts eye-catching maiden WA niobium resource',
  'summary': 'Hastings Technology Metals has revealed a maiden 6.7 million-tonne niobium resource estimate in WA, grading 0.23 per cent for 15,501 tonnes of contained metal.'},
 {'title': 'Pressure on the Fed is mounting as sharemarkets spiral again',
  'summary': 'Nearly a month ago, Wall Street plunged in response to a shock set of US job figures. Overnight, the market plunged again, in expectatio